In [13]:
import sys
sys.path.append('../../')
import pandas as pd
import requests
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.stattools import adfuller
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import functions.functions as base_fx
import functions.portfolio_functions as portfolio_fx
import functions.pair_portfolio_functions as pairs_portfolio_fx
import functions.mean_algos as algos
import importlib
from binance.client import Client,BinanceAPIException
import json
import math

In [14]:
symbols = ['XRPBTC']# ['ETHBTC', 'XRPBTC', 'XLMBTC', 'EOSBTC']


In [49]:
quotes = pd.DataFrame()
#start = '2011-06-1' ## Starts with non-correlated period (drops 40% at the start)
start = '2020-06-01'
end = '2020-06-02'
client = Client('NEI4wgKwWXw8uEjqyHPuBKFgKPgv4xPdpeey2Z17Aek8YWyaOHTsEEAUWPVyl4WH', 'UCw65kdS9N4GkJt2WpQTYgFkXPB6rMkM1ETQARfIbBHL1836Q5oX08IuAUWzFNbD')
for s in symbols:
    print(s)
    klines = client.get_historical_klines(s, Client.KLINE_INTERVAL_1MINUTE, "1 Jun, 2020", "04 Jun, 2020")
    df = pd.DataFrame(klines)
    #print(df.head(1))
    if len(quotes) == 0:
        quotes['Date'] = df[0]
        quotes[s] = pd.to_numeric(df[4])
    else:
        new_df = pd.DataFrame()
        new_df['Date'] = df[0]
        new_df[s] = pd.to_numeric(df[4])
        quotes = quotes.merge(new_df, how="inner", on="Date")
    #print(quotes)
#print(quotes.tail(20))

XRPBTC


In [17]:
quotes['Date'] = pd.to_datetime(quotes['Date'], unit='ms')
quotes.set_index('Date', inplace=True)
print(quotes)

                       XRPBTC
Date                         
2020-08-01 00:00:00  0.000023
2020-08-01 00:01:00  0.000023
2020-08-01 00:02:00  0.000023
2020-08-01 00:03:00  0.000023
2020-08-01 00:04:00  0.000023
2020-08-01 00:05:00  0.000023
2020-08-01 00:06:00  0.000023
2020-08-01 00:07:00  0.000023
2020-08-01 00:08:00  0.000023
2020-08-01 00:09:00  0.000023
2020-08-01 00:10:00  0.000023
2020-08-01 00:11:00  0.000023
2020-08-01 00:12:00  0.000023
2020-08-01 00:13:00  0.000023
2020-08-01 00:14:00  0.000023
2020-08-01 00:15:00  0.000023
2020-08-01 00:16:00  0.000023
2020-08-01 00:17:00  0.000023
2020-08-01 00:18:00  0.000023
2020-08-01 00:19:00  0.000023
2020-08-01 00:20:00  0.000023
2020-08-01 00:21:00  0.000023
2020-08-01 00:22:00  0.000023
2020-08-01 00:23:00  0.000023
2020-08-01 00:24:00  0.000023
2020-08-01 00:25:00  0.000023
2020-08-01 00:26:00  0.000023
2020-08-01 00:27:00  0.000023
2020-08-01 00:28:00  0.000023
2020-08-01 00:29:00  0.000023
...                       ...
2020-08-03

In [22]:
filtered = quotes#[quotes.index >= '2020-07-01']
filtered = filtered * 100000
normalized = filtered / filtered.iloc[0]
normalized.dropna(inplace=True)

In [45]:
def get_settings():
    return {
        'symbol': 'XRPBTC',
        'limit': 0,
        'leverage_limit': 0,
        'max_leverage': 1,
        'fee': 0.00, #0.1%,
        'stop_loss': -0.002, ## already in pct
        'take_profit': 0.01 
       }

In [46]:
importlib.reload(base_fx)
importlib.reload(portfolio_fx)
importlib.reload(algos)
settings = get_settings()
portfolio, indicators = algos.scalping_macd(settings, filtered)

0.0
-0.0004378283712785924
-0.0017513134851140366
-0.003940455341506333
Trigger SL on s1: 2020-08-01 00:03:00 -0.003940455341506333
0.0
0.0
0.0013169446883231073
0.0030728709394205467
0.0013169446883231073
0.0008779631255487752
0.0021949078138717715
0.0008779631255487752
0.0017559262510974394
0.0013169446883231073
0.0026338893766463256
0.0
-0.0017559262510975504
-0.0017559262510975504
-0.0013169446883232183
-0.0013169446883232183
-0.0004389815627743321
-0.0004389815627743321
-0.0021949078138716605
Trigger SL on s1: 2020-08-01 00:22:00 -0.0021949078138716605
0.0
-0.0008752735229760056
0.0
-0.00043763676148789177
-0.0008752735229760056
-0.0017505470459519001
-0.0013129102844638973
-0.002188183807440014
Trigger SL on s1: 2020-08-01 00:49:00 -0.002188183807440014
0.0
-0.0004366812227073469
0.0004366812227074579
0.0004366812227074579
0.0008733624454149158
0.0013100436681221517
0.0004366812227074579
0.0013100436681221517
0.0013100436681221517
0.0026200873362445254
0.0013100436681221517
0.002

0.0
-0.002810116419108888
Trigger SL on s1: 2020-08-02 02:15:00 -0.002810116419108888
0.0
0.000401929260450018
-0.002813504823151014
Trigger SL on s1: 2020-08-02 02:18:00 -0.002813504823151014
0.0
-0.0004017677782240936
-0.0008035355564482982
-0.0004017677782240936
-0.0028123744475693213
Trigger SL on s1: 2020-08-02 02:23:00 -0.0028123744475693213
0.0
-0.045141874462596654
Trigger SL on s1: 2020-08-02 04:42:00 -0.045141874462596654
0.0
-0.03982649842271302
Trigger SL on s1: 2020-08-02 04:44:00 -0.03982649842271302
0.0
-0.0022753128555175195
Trigger SL on s1: 2020-08-02 09:47:00 -0.0022753128555175195
0.0
0.0011385199240987465
0.0011385199240987465
0.003795066413662118
0.004933586337760865
0.00189753320683117
0.005313092979126965
0.004174573055028441
0.0015180265654646252
0.0
0.0026565464895633717
0.00189753320683117
0.004554079696394542
0.004933586337760865
0.003795066413662118
0.0011385199240987465
0.0011385199240987465
0.0011385199240987465
0.0015180265654646252
0.0015180265654646252

0.0
-0.0003617945007237111
0.0018089725036178894
-0.0028943560057888007
Trigger SL on s1: 2020-08-03 20:24:00 -0.0028943560057888007
0.0
-0.001826817683595161
-0.003653635367190322
Trigger SL on s1: 2020-08-03 20:33:00 -0.003653635367190322
0.0
0.0010893246187364536
0.0010893246187364536
0.004357298474945592
0.005446623093681935
0.004357298474945592
0.011619462599854802
Trigger TP on s1: 2020-08-03 22:55:00 0.011619462599854802
0.0
-0.001808318264014619
-0.0021699819168173873
Trigger SL on s1: 2020-08-03 23:55:00 -0.0021699819168173873


In [47]:
pairs_portfolio_fx.plot_objects([{'label': 'nav', 'value': portfolio['nav']['nav']}])

In [48]:
print(portfolio['history'])

    symbol  type      amount  bought_at           bought_on status trigger_sl  \
0   XRPBTC   BUY  100.000000      2.284 2020-08-01 00:00:00  CLOSE       True   
1   XRPBTC  SELL   99.605954      2.278 2020-08-01 00:04:00  CLOSE       True   
2   XRPBTC   BUY   99.387329      2.285 2020-08-01 00:42:00  CLOSE       True   
3   XRPBTC   BUY   99.169851      2.290 2020-08-01 01:55:00  CLOSE       True   
4   XRPBTC  SELL   98.606878      2.272 2020-08-01 02:43:00  CLOSE       True   
5   XRPBTC  SELL   98.346472      2.280 2020-08-01 02:48:00  CLOSE       True   
6   XRPBTC   BUY   98.001397      2.288 2020-08-01 03:27:00  CLOSE       True   
7   XRPBTC  SELL   99.072217      2.287 2020-08-01 05:56:00  CLOSE       True   
8   XRPBTC  SELL   98.855618      2.294 2020-08-01 06:00:00  CLOSE       True   
9   XRPBTC   BUY   99.932946      2.288 2020-08-01 09:11:00  CLOSE       True   
10  XRPBTC  SELL  100.937517      2.298 2020-08-01 13:13:00  CLOSE       True   
11  XRPBTC  SELL  100.717896